In [31]:
'''

- 데이터는 과거 보고서 작성때 사용하던 데이터와 다를 수 있으나, 데이터 형태 및 전처리과정은 동일합니다.
- 형태소 분석은 소요시간이 많은 관계로 500개의 데이터(기사)만 사용했습니다.

'''

#필요 패키지
import pandas as pd
import numpy as np
import os
import re
import time
from tqdm import tqdm
import datetime
from konlpy.tag import Okt
import string
import glob
from ckonlpy.tag import Postprocessor
from ckonlpy.tag import Twitter

In [32]:
## ① 보유장비 성능 고려, 일별 20%를 추출한 총 16만여 건의 분석대상 뉴스DB 구축 ##
data_url = pd.read_excel('C:/Users/USER/Desktop/전처리정리/1월~8월URL.xlsx') # 1분정도 걸림

def sampling_func(data777, sample_pct):
    np.random.seed(123)
    N = len(data777)
    sample_n = int(len(data777)*sample_pct) # integer
    sample = data777.take(np.random.permutation(N)[:sample_n])
    return sample
data_url_2 = data_url.groupby('RGSDE',group_keys=False).apply(sampling_func,sample_pct=0.2) #일자별 샘플링

print(data_url_2.groupby(data_url['RGSDE']).size())

RGSDE
20200101     223
20200102     470
20200103      56
20200106     136
20200107     431
            ... 
20200825    1563
20200826    1556
20200827    1600
20200828    1459
20200829     508
Length: 237, dtype: int64


In [34]:
print('수정전 url 수 : {}개 '.format(len(data_url)))
print('수정후 url 수 : {}개 '.format(len(data_url_2)))

수정전 url 수 : 982492개 
수정후 url 수 : 196407개 


In [35]:
#이후 크롤링 실행  데이터 저장

#데이터 형태
data = pd.read_excel('C:/Users/USER/Desktop/전처리정리/data0928.xlsx', index_col=0)
data = data.iloc[0:500,0:5] #빠른 실행을 위해 500개의 데이터만 이용

data

,date,title,text,category,url
0,2020-03-01,코로나19에 3·1절 기념식 대폭 축소,코로나19 확산으로 오늘 열리는 101주년 삼일절 기념식이 최소 인원으로 치러집니다...,사회,https://news.naver.com/main/read.nhn?mode=LSD&...
1,2020-03-01,"삼척시, 감염병 예방과 정부사업 선점 투 트랙 대응",정부 공모사업 지난해보다 20% ↑ 1641억 발굴 [홍춘봉 기자(=삼척)] 삼척시...,사회,https://news.naver.com/main/read.nhn?mode=LSD&...
2,2020-03-01,신천지 이만희 도대체 어디있나…코로나 확산 책임 '대국민사과' 할까,대구 지역 코로나19 확진자 급증…신천지 연관신천지 교주 이만희 어디있나 두문불출박...,사회,https://news.naver.com/main/read.nhn?mode=LSD&...
3,2020-03-01,[경제전망]6.2兆 이상 코로나19 추경안 나온다…물가 변화도 주목,[이미지출처=연합뉴스][아시아경제 김현정 기자] 신종 코로나바이러스감염증(코로나19...,경제,https://news.naver.com/main/read.nhn?mode=LSD&...
4,2020-03-01,코로나19에도 2월 수출 4.5% 반등…대중 수출 부진 등 악재 가시화,[CBS노컷뉴스 김선경 기자]수출이 코로나19 악재에도 불구하고 15개월 만에 증가...,경제,https://news.naver.com/main/read.nhn?mode=LSD&...
...,...,...,...,...,...
495,2020-03-02,[포착] 이만희 “조용합시다”… 퇴장하면서 ‘엄지 척’,신천지 교주 이만희씨가 2일 오후 기자회견을 마치고 나서고 있다. 이하 윤성호 기자...,사회,https://news.naver.com/main/read.nhn?mode=LSD&...
496,2020-03-02,정부 “중증환자 수용 거부 지자체 처벌”,“환자이송 국립의료원 통제 따라야 / 전국 병상은 국가적 자산” 강조 / 일부 시·...,사회,https://news.naver.com/main/read.nhn?mode=LSD&...
497,2020-03-02,[코로나19] 국내 누적 확진자 4335명…하루새 599명 추가,[아이뉴스24 권준영 기자] 국내 신종 코로나바이러스 감염증(코로나19) 확진자가 ...,사회,https://news.naver.com/main/read.nhn?mode=LSD&...
498,2020-03-02,"미스터빠삭 사업설명회 시식 중단, '치즈돈까스 끝판왕' 호평, 코돈브루 신메뉴 출시",[엑스포츠뉴스 김지연 기자] 소자본 돈까스 프랜차이즈 미스터빠삭은 지난 28일 상암...,경제,https://news.naver.com/main/read.nhn?mode=LSD&...


In [36]:
## ② 형태소 분석 후 축약어·이음동의어(예: 유연근무 → 유연근무제) 등 자연어 처리 ##
## ③ 파이썬 자연어 처리 패키지 내 형태소 사전(NLP)에 신규 단어(명사)들 추가 ##

# 신규 단어 추가
twi = Twitter()
add_noun = ['비대면','확진자','거리두기','사랑제일교회','의료기관','의료진','집단감염','유연근무','유연근무제','유연근로제','한국판뉴딜','디지털뉴딜',
            '그린뉴딜','휴먼뉴딜','취약계층','긴급재난지원금','고용보험','진료거부','택배기사','선별진료소','고용보험','특수고용직','특고','재배포',
            '고용안정지원금','공유오피스','공용오피스','저소득층','실업급여','유통업계','물류센터','소상공인','유연근로','특수형태근로자','특수형태근로종사자']
eco_word = pd.read_excel('C:/Users/USER/Desktop/전처리정리/★주요키워드 및 배제어.xlsx')
eco_keyword = list(eco_word['경제키워드'])
for i in eco_keyword:
    add_noun.append(i)

for i in add_noun:
    twi.add_dictionary(i,'Noun') #새로운 단어 등록 하기

    
    
#통합어 만들기
replace = {'유연근로제':'유연근무제','유연근무':'유연근무제','유연근로':'유연근무제','특고':'특수고용직','특수형태근로자':'특수고용직','특수형태근로종사자':'특수고용직'}
postprocessor = Postprocessor(base_tagger=twi, replace=replace, passtags={'Noun'})


#단어 등록 확인하기
postprocessor.pos('확진자와 비대면으로 거리두기를 사랑제일교회에서부터 전광훈이 사전조사 의료기관과 의료진이 집단감염이 걸려 집단감염, 재택근무하고 '
          '유연근무와 유연근무제를 하고 지역경제와 한국판뉴딜과 그린뉴딜과 디지털뉴딜을 휴면뉴딜하고 취약계층의 긴급재난지원금을 특수고용직에게'
          '고용보험을 진료거부하며 택배기사는 온라인수업으로 선별진료소로 간다 소상공인 갭투자를 계액갱신공매공매도공시가격과징금과학기술괴리율국가경쟁력국내총생산'
          '국민건강보험국민연금국제원유가격규제지역그린뉴딜그린벨트금리금리인하금융기관금융세제금융위기금융지원배달서비스배당금배터리백신보유세보이스피싱부동산부동산시장'
          '분양가분양가상한제불법사금융블록체인비상경제비상경제회의빅데이터사모펀드사이드카사회적거리두기산유국상장주식상장지수증권상장지수펀드생필품서킷브레이커선물세트선물환세금'
          '세입자셧다운소비심리소상공인손소독제수소차수출규제스마트스토어스마트팜스타트업시장심리지수신용대출신용등급신용카드신재생에너지실물경제실업자안전자산액화천연가스양도세양도소득양도소득세'
          '양도차익양적완화에너지저장시스템오픈뱅킹온라인쇼핑몰외환시장외환위기용적률우선주운전자보험원격수업원격의료원달러환율위안화 AI에서 특수형태근로종사자와 특수형태근로자와 특고')



C:\Users\USER\anaconda3\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


[('확진자', 'Noun'),
 ('비대면', 'Noun'),
 ('거리두기', 'Noun'),
 ('사랑제일교회', 'Noun'),
 ('전광훈', 'Noun'),
 ('사전', 'Noun'),
 ('조사', 'Noun'),
 ('의료기관', 'Noun'),
 ('의료진', 'Noun'),
 ('집단감염', 'Noun'),
 ('집단감염', 'Noun'),
 ('재택근무', 'Noun'),
 ('유연근무제', 'Noun'),
 ('유연근무제', 'Noun'),
 ('지역', 'Noun'),
 ('경제', 'Noun'),
 ('한국판뉴딜', 'Noun'),
 ('그린뉴딜', 'Noun'),
 ('디지털뉴딜', 'Noun'),
 ('휴면', 'Noun'),
 ('뉴딜', 'Noun'),
 ('취약계층', 'Noun'),
 ('긴급재난지원금', 'Noun'),
 ('특수고용직', 'Noun'),
 ('고용보험', 'Noun'),
 ('진료거부', 'Noun'),
 ('택배기사', 'Noun'),
 ('온라인수업', 'Noun'),
 ('선별진료소', 'Noun'),
 ('간다', 'Noun'),
 ('소상공인', 'Noun'),
 ('갭투자', 'Noun'),
 ('계액', 'Noun'),
 ('갱신', 'Noun'),
 ('공매', 'Noun'),
 ('공매도', 'Noun'),
 ('공시가격', 'Noun'),
 ('징금', 'Noun'),
 ('과학기술', 'Noun'),
 ('괴리율', 'Noun'),
 ('국가경쟁력', 'Noun'),
 ('국내총생산', 'Noun'),
 ('국민건강보험', 'Noun'),
 ('국민연금', 'Noun'),
 ('국제원유가격', 'Noun'),
 ('규제지역', 'Noun'),
 ('그린뉴딜', 'Noun'),
 ('그린벨트', 'Noun'),
 ('금리', 'Noun'),
 ('금리인하', 'Noun'),
 ('금융기관', 'Noun'),
 ('금융세제', 'Noun'),
 ('금융위기', 'Noun'),
 ('금융지

In [37]:
## ④ 분석목적에 맞지 않는 단어(명사)들은 배제어로 등록 ##

#형태소 분석 하기
words = [[j[i][0] for i in range(len(j))] for j in [postprocessor.pos(i) for i in tqdm(list(data['text']))]]

#배제어 등록하기
Prohibit_words = ['기자','연합뉴스','뉴시스','시사저널','신문','뉴스','사진','헤럴드경제','노컷뉴스','파이낸셜뉴스','특파원',
                  '라며','대해','지난','위해','오전','오후','무단','배포','이데일리','머니투데이','앵커','지금','때문','이번',
                  '통해','정도','경우','관련','이미지','출처','일보','바로가기','까지','여개','도록','이나','재배포','처럼','면서',
                  '거나','이제','지난달','어요']
for i in list(eco_word['배제어']):
    Prohibit_words.append(i)
    
#배제어 제거
j = 0
for i in tqdm(words):
    for k in Prohibit_words:
        while k in i:
            i.remove(k)
    words[j] = i
    j += 1 #불용어 제외

100%|██████████████████████████████████████████████████████████████████████████████| 500/500 [00:00<00:00, 1343.95it/s]


In [38]:
## ⑤ 기본키워드(Seed words) 외 2자리 이하 단어 제외(의미없는 키워드 배제) ##
for k in range(len(words)):
    words[k] = [i for i in words[k] if len(i) > 2 or i in 
                ['배달','택배','고용','취업','실업','채용','구직','청년','유통','해고']]  


data['words'] = words


In [40]:
## ⑥ 본문 단어 50개 이하 뉴스 (속보, 포토 뉴스 등)는 분석에서 제외 ##

data['length_word'] = [len(i) for i in tqdm(data['words'])] #단어 갯수 세기

data50 = data[data['length_word'] >= 50]  #단어 갯수 50개 이하 제거

data50 = data50.reset_index(drop = True)

data50


data50.to_excel('C:/Users/USER/Desktop/전처리정리/data50.xlsx',header=True) #저장하기

100%|████████████████████████████████████████████████████████████████████████████| 500/500 [00:00<00:00, 501831.06it/s]
